In [1]:
# Load the dataset
with open("semeval_test_set_raw.txt", "r") as f:
    data = f.read()

In [3]:
# Create Active Passive Dataset
################################
cur_pos = 0
"""
    Returns the first occurance of a sentence at or after start_index
"""
def get_next_sentence(start_index):
    sent_start = data.find('\"', start_index) + 1
    sent_end = data.find('\"', sent_start)
    return data[sent_start:sent_end], sent_end

demo_sent, index = get_next_sentence(0)
print(demo_sent)

The most common <e1>audits</e1> were about <e2>waste</e2> and recycling.


In [5]:
"""
Removes the <e1> </e2> tags from the sentence.
"""
def remove_sent_tags(sent):
    sent = sent.replace("<e1>", "")
    sent = sent.replace("</e1>", "")
    sent = sent.replace("<e2>", "")
    return sent.replace("</e2>", "")

demo_sent = remove_sent_tags(demo_sent)
print(demo_sent)

The most common audits were about waste and recycling.


In [6]:
"""
returns the first label occuring after 
"""
def get_next_label(start_index):
    # The label occurs one character after the newline
    label_start = data.find("\n", start_index) + 1
    label_end = data.find("\n", label_start)
    return data[label_start:label_end], label_end
demo_label, index = get_next_label(index)
print(demo_label, index)

8002	"The <e1>company</e1> fabricates plastic <e2>chairs</e2>." 143


In [7]:
def ignore_next_comment(start_index):
    return data.find("\n\n", start_index)
index = ignore_next_comment(index)
print(index)

-1


In [8]:
label_map = {}
label_map["Component-Whole"] = 0

"""
Returns active or passive based on e2,e1 in label and returns the label itself
"""
def parse_label(label):
    e1_pos = label.find("e1")
    e2_pos = label.find("e2")
    if e1_pos == -1:
        return "", 0
    is_active = int(e1_pos < e2_pos)
    end_label_pos = label.find("(")
    return label[:end_label_pos], is_active

demo_label, active = parse_label(demo_label)
print(demo_label, active)

8002	"The <e1>company</e1> fabricates plastic <e2>chairs</e2>. 1


In [9]:
"""
    ACTIVE OR PASSIVE
    returns a cleaned sentence and its active or passive label
    this is not exactly active or passive
"""
def extract_data_point(start_index):
    sent, index = get_next_sentence(start_index)
    sent = remove_sent_tags(sent)
    label, index = get_next_label(index)
    label, is_active = parse_label(label)
    index = ignore_next_comment(index)
    if label == "":
        return 0, "", index
    return is_active, sent, index

In [50]:
index = 0
#Contains sent and label in dict this is not exactly active or passive
data_list = []

"""
-3 since the last comment is followed by two \n\ns
"""
while index <= len(data)-3:
    data_dict = {}
    is_active, sent, index = extract_data_point(index)
    
    if sent == "":
        continue;
    
    data_dict["sent"] = sent
    data_dict["label"] = is_active
    data_list.append(data_dict)
    
print(len(data))
print(len(data_list))
print(sent_list[:3])

1365759
6584
['The system as described above has its greatest application in an arrayed configuration of antenna elements.', 'The child was carefully wrapped and bound into the cradle by means of a cord.', 'The author of a keygen uses a disassembler to look at the raw assembly code.']


In [51]:
print(data_list[137])

{'sent': 'Initially the user calibrates the microscope with a fluorescence lifetime standard positioned at the sample plane.', 'label': 0}


In [67]:
# store the data in a file
##################
import json
with open("semeval_dataset.json", "w") as f:
    data = json.dump(data_list, f, sort_keys=False, indent=4)

In [31]:
import os
os.getcwd()


'C:\\Users\\kaidpinck\\thesis\\thesis\\classifier\\semeval2010task8'

In [10]:
"""
returns the sentence along with its semantic label
"""
def extract_semeval_data(start_index):
    sent, index = get_next_sentence(start_index)
    sent = remove_sent_tags(sent)
    label, index = get_next_label(index)
    label, _ = parse_label(label)
    index = ignore_next_comment(index)
    return label, sent, index

In [11]:
index = 0
#dataset all labels
#################################################################

label_map = {}
label_map["Cause-Effect"] = 0
label_map["Instrument-Agency"] = 1
label_map["Product-Producer"] = 2
label_map["Content-Container"] = 3
label_map["Entity-Origin"] = 4
label_map["Entity-Destination"] = 5
label_map["Component-Whole"] = 6
label_map["Member-Collection"] = 7
label_map["Message-Topic"] = 8
label_map[""] = 9



data_list = []

"""
-3 since the last comment is followed by two \n\ns
"""
while index <= len(data)-3:
    data_dict = {}
    label, sent, index = extract_semeval_data(index)
    
    data_dict["sent"] = sent
    data_dict["label"] = label_map[label]
    data_list.append(data_dict)
    
print(len(data_list))
print(sent_list[:3])
print(data_list[:5])

KeyError: '8002\t"The <e1>company</e1> fabricates plastic <e2>chairs</e2>.'

In [77]:
index = 0
#dataset only cause-effect and other
#################################################################

label_map = {}
label_map["Cause-Effect"] = 0
label_map["Instrument-Agency"] = 1
label_map["Product-Producer"] = 1
label_map["Content-Container"] = 1
label_map["Entity-Origin"] = 1
label_map["Entity-Destination"] = 1
label_map["Component-Whole"] = 1
label_map["Member-Collection"] = 1
label_map["Message-Topic"] = 1
label_map[""] = 1



data_list = []

"""
-3 since the last comment is followed by two \n\ns
"""
while index <= len(data)-3:
    data_dict = {}
    label, sent, index = extract_semeval_data(index)
    
    data_dict["sent"] = sent
    data_dict["label"] = label_map[label]
    data_list.append(data_dict)
    
print(len(data_list))
print(sent_list[:3])
print(data_list[:5])

8000
['The system as described above has its greatest application in an arrayed configuration of antenna elements.', 'The child was carefully wrapped and bound into the cradle by means of a cord.', 'The author of a keygen uses a disassembler to look at the raw assembly code.']
[{'sent': 'The system as described above has its greatest application in an arrayed configuration of antenna elements.', 'label': 1}, {'sent': 'The child was carefully wrapped and bound into the cradle by means of a cord.', 'label': 1}, {'sent': 'The author of a keygen uses a disassembler to look at the raw assembly code.', 'label': 1}, {'sent': 'A misty ridge uprises from the surge.', 'label': 1}, {'sent': 'The student association is the voice of the undergraduate student population of the State University of New York at Buffalo.', 'label': 1}]


In [78]:
# store the data in a file
##################
import json
with open("causalclassifier_dataset.json", "w") as f:
    data = json.dump(data_list, f, sort_keys=False, indent=4)